# Lab 9

— Generative Adversarial Networks (GANs)

Build and train a **vanilla GAN** using the exact building blocks shown in the reference script:

- `9-generative_adversarial_network.ipynb`
  and concepts from the slides: `9 - GANs.pptx`.

Each task below includes **explicit steps**, **where to find matching content** in the script, and the **expected output**.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/username/repo/blob/main/path-to-file)  
**Students:** Replace `username`, `repo`, and `path-to-file` with your own GitHub username, repository name, and the path to this file.  
After opening in Colab, go to **File → Save a copy to GitHub** (your repo) before editing.

#### NOTE: You will need to use a GPU on Google Colab for this to run in a reasonable amount of time without memory running out. I suggest using A100 GPU. Edit -> Notebook Settings


## H.1 Data Preparation — MNIST 28×28 Grayscale

**Reference script:** `9-generative_adversarial_network.ipynb`


**Where to find in the script:**

- Imports and dataset load → near the top cells (look for `keras.datasets.mnist.load_data()` and normalization step).
- Reshaping to `(N, 28, 28, 1)` and scaling to `[0,1]` → the preprocessing cell after load.


In [ ]:
# TODO (H.1): Load MNIST, scale to [0,1], and reshape to (N,28,28,1).
# Steps:
# 1) Use keras.datasets.mnist.load_data()
# 2) Convert to float32 and divide by 255.0
# 3) Expand dims to add channel axis
# Output variables: x_train (float32, shape=(60000,28,28,1))
pass


**Expected output (H.1):**

- `x_train.shape == (60000, 28, 28, 1)`
- `x_train.min() >= 0.0` and `x_train.max() <= 1.0`


## H.2 Define the Discriminator

**Reference script:** `9-generative_adversarial_network.ipynb`


**Where to find in the script:**

- Discriminator block → look for a `Sequential` with Conv2D → LeakyReLU → Dropout stacks and a final Dense(1,'sigmoid').
- Optimizer setup for discriminator (often Adam with β1≈0.5).


In [ ]:
# TODO (H.2): Build the discriminator.
# Architecture (match script style):
#   Input: (28,28,1)
#   Conv2D(64, kernel_size=5, strides=2, padding='same') → LeakyReLU(0.2) → Dropout(0.3)
#   Conv2D(128, kernel_size=5, strides=2, padding='same') → LeakyReLU(0.2) → Dropout(0.3)
#   Flatten → Dense(1, activation='sigmoid')
# Compile with loss='binary_crossentropy', optimizer=Adam(lr ~ 2e-4, beta_1 ~ 0.5), metrics=['accuracy']
# Name your model variable: discriminator
pass


**Expected output (H.2):**

- `discriminator` is a compiled Keras model.
- Calling `discriminator(tf.zeros([1,28,28,1]))` returns a (1,1) tensor in [0,1].


## H.3 Define the Generator

**Reference script:** `9-generative_adversarial_network.ipynb`


**Where to find in the script:**

- Generator block → look for Dense → Reshape → Conv2DTranspose stacks finishing with sigmoid.
- Latent vector size (e.g., `latent_dim = 100`).


In [ ]:
# TODO (H.3): Build the generator that maps z∼N(0,1) of shape (latent_dim,) to (28,28,1).
# Suggested pattern (match script):
#   Dense(7*7*128) → LeakyReLU → Reshape(7,7,128)
#   Conv2DTranspose(64, kernel_size=5, strides=2, padding='same') → LeakyReLU
#   Conv2DTranspose(1,  kernel_size=5, strides=2, padding='same', activation='sigmoid')
# Name your model variable: generator
pass


**Expected output (H.3):**

- `generator` is a Keras model that, given a (batch, latent_dim) noise input, outputs images of shape (batch,28,28,1) in [0,1].


## H.4 Adversarial (GAN) Model

**Reference script:** `9-generative_adversarial_network.ipynb`


**Where to find in the script:**

- Code that freezes the discriminator (`discriminator.trainable = False`) and builds a `gan = Model(z, discriminator(generator(z)))`.
- Optimizer for GAN (often same Adam params).


In [ ]:
# TODO (H.4): Build the GAN by stacking generator → discriminator.
# Steps:
# 1) Set discriminator.trainable = False (for the GAN compile step)
# 2) Create an Input for z (latent_dim,)
# 3) Pass through generator then discriminator
# 4) Compile GAN with loss='binary_crossentropy' and Adam(lr ~ 2e-4, beta_1 ~ 0.5)
# Name your model variable: gan
pass


**Expected output (H.4):**

- `gan` is a compiled model that takes noise and returns a real/fake score.


## H.5 Training Loop (Memory-aware)

**Reference script:** `9-generative_adversarial_network.ipynb`


**Where to find in the script:**

- The custom loop alternating discriminator and generator updates (often using real labels=1 and fake labels=0).
- Epoch logging and periodic sampling of generated images.


In [ ]:
# TODO (H.5): Implement the training loop.
# Memory-safe defaults:
#   BATCH = 64 on strong GPUs, otherwise start at 32 or 16. If OOM, reduce further.
#   EPOCHS = 30 (OK for class); to speed up, set 10–15.
# Steps per epoch ~ len(x_train)//BATCH.
# Steps per training step:
#   a) Sample real batch from x_train → label=1
#   b) Sample noise, generate fake batch → label=0
#   c) Train discriminator on real then on fake; average the two losses.
#   d) Train GAN (freeze D) with noise and label=1 (generator tries to fool D).
#   e) Every N epochs: generate 4×4 image grid.

# IMPORTANT: Experiment with epochs and other hyperparameters to try to get the best results.
pass


**Expected output (H.5):**

- Per-epoch logs: D loss/acc and GAN (generator) loss.
- A few image grids showing samples improving over time.
- Final print of losses across epochs.


## Discussion Questions


1. Why can GAN training be unstable? Name two symptoms and one stabilization trick.
2. What happens if the discriminator is much stronger than the generator? How would you adjust training?
3. If generated digits are blurry, which change to the generator would you try first, and why?
